In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import cut_the_tails as ct
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('../data_sets\\bike_sharing_hour.csv')
target = 'cnt'

0         16
1         40
2         32
3         13
4          1
        ... 
17374    119
17375     89
17376     90
17377     61
17378     49
Name: cnt, Length: 17379, dtype: int64

In [3]:
features = ['season',
 'mnth',
 'hr',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed']

In [4]:
fig = px.histogram(df, x=target, nbins=40)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

# Cauda Ótima

In [10]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

model = RandomForestRegressor(max_depth=5, random_state=0) 
classifier = RandomForestClassifier(max_depth=5, random_state=0)

x, fval = ct.get_cuts_direct_optimization(df, target, features, classifier, model, 'brute')

x,fval


[0. 0.] 0.42729335164648646
[0.   0.05] 0.42728100420640225
[0.  0.1] 0.4260955297887463
[0.   0.15] 0.41542874872430896
[0.  0.2] 0.4212797879273013
[0.   0.25] 0.43439168540805967
[0.  0.3] 0.43333169293023394
[0.   0.35] 0.4120066326302258
[0.  0.4] 0.4199357831582639
[0.   0.45] 0.40893921158871327
[0.  0.5] 0.41831581030117465
[0.   0.55] 0.42993785469425627
[0.  0.6] 0.451062193171366
[0.   0.65] 0.48069225215075695
[0.  0.7] 0.5121666154289599
[0.   0.75] 0.539759309759995
[0.  0.8] 0.5237290155641448
[0.   0.85] 0.5187713648003379
[0.  0.9] 0.5076884557043395
[0.   0.95] 0.45703980476735695
[0.   0.05] 0.42728100420640225
[0.05 0.05] 0.42636712975528485
[0.05 0.1 ] 0.4296896688042839
[0.05 0.15] 0.4367919373912196
[0.05 0.2 ] 0.42497980257467133
[0.05 0.25] 0.42489459761410003
[0.05 0.3 ] 0.42747606025093293
[0.05 0.35] 0.41437667561266295
[0.05 0.4 ] 0.4124070261178746
[0.05 0.45] 0.40276529068162487
[0.05 0.5 ] 0.41730073280918045
[0.05 0.55] 0.4266601528987877
[0.05 0.6 ] 0.

(array([0.05015625, 0.4528125 ]), 0.4022394748805212)

In [ ]:
cdf = ct.split_by_quantile_class(df,target,[0.0,0.5])

In [ ]:
X = cdf[features].to_numpy()
y_tail = cdf['tail_class'].to_numpy()
y = cdf[target].to_numpy()

###Using split###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train_aux, X_test_aux, y_train_tail, y_test_tail = train_test_split(X, y_tail, test_size=0.2, random_state=0)

In [ ]:
baseline = RandomForestRegressor(max_depth=5, random_state=0)
#baseline = XGBRegressor(n_estimators=100, max_leaves=0, random_state = 0)
#baseline = DecisionTreeRegressor(random_state=0, max_depth=5) 

In [ ]:
#tail_classifier = ct.fit_tail_classifier(X,y_tail,RandomForestClassifier(max_depth=5, random_state=0))
#tail_classifier = ct.fit_tail_classifier(X,y_tail,XGBClassifier(n_estimators=100, max_leaves=0, random_state = 0))
#tail_classifier = ct.fit_tail_classifier(X,y_tail,DecisionTreeClassifier(random_state=0, max_depth=5)) 

###Using split###
tail_classifier = ct.fit_tail_classifier(X,y_tail,RandomForestClassifier(max_depth=5, random_state=0))
#tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,XGBClassifier(n_estimators=100, max_leaves=0, random_state = 0))
#tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,DecisionTreeClassifier(random_state=0, max_depth=5)) 

In [ ]:
#models = ct.fit_tail_models(X,y,y_tail,baseline)

###Using split###
models = ct.fit_tail_models(X_train,y_train,y_train_tail,baseline)

In [ ]:
#baseline.fit(X,y)
#y_base = baseline.predict(X)
#y_tail = ct.batch_tail_predict(X,tail_classifier,models)

###Using split###
baseline.fit(X_train,y_train)
y_base = baseline.predict(X_test)
y_tail = ct.batch_tail_predict(X_test,tail_classifier,models)

In [ ]:
#print(mean_absolute_error(y_base,y))
#print(mean_absolute_error(y_tail,y))
#print(mean_absolute_error(y,y))

###Using split###
print(mean_absolute_error(y_base,y_test))
print(mean_absolute_error(y_tail,y_test))
print(mean_absolute_error(y_test,y_test))

In [ ]:
#print(mean_absolute_percentage_error(y_base,y))
#print(mean_absolute_percentage_error(y_tail,y))
#print(mean_absolute_percentage_error(y,y))

###Using split###
print(mean_absolute_percentage_error(y_base,y_test))
print(mean_absolute_percentage_error(y_tail,y_test))
print(mean_absolute_percentage_error(y_test,y_test))